In [1]:
import pandas as pd
import nltk as nltk
import sklearn
from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
import tensorflow as tf
import keras
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer,SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
import numpy as np
import html
import re

from IPython.display import clear_output
import math


Using TensorFlow backend.


In [2]:
import pickle

In [3]:
pd.set_option(
    'display.max_colwidth', 0
)
np.random.seed(42)

In [4]:
nltk.download('subjectivity')
nltk.download('vader_lexicon')


[nltk_data] Downloading package subjectivity to
[nltk_data]     /Users/ekaterinaromanovskaya/nltk_data...
[nltk_data]   Unzipping corpora/subjectivity.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ekaterinaromanovskaya/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [6]:
# df = pd.read_csv('first100k.csv')
df = pd.read_csv('hacker_news_sample.csv',nrows=100000)

In [7]:
def preprocess_df(df):
    df = df[df['text'].notna()]
    df['neg'], df['neu'], df['pos'], df["compound"], df["subjectivity"] = [np.nan, np.nan,np.nan,np.nan,np.nan]    
    return df

def add_sentiment_subj(df):
    sid = SentimentIntensityAnalyzer()
    for i, row in df.iterrows():
        text = row["text"]
        text = html.unescape(text)
        text =  re.sub('<[^<]+?>', '', text)
        df.at[i,"text"] = text
        #subj = nltk.sentiment.util.demo_sent_subjectivity(text)
        #df.at[i,"subjectivity"] = subj
        ss = sid.polarity_scores(text)
        for k in ss:
            df.at[i,k] = ss[k]
        
    



In [16]:
pipeline = Pipeline(
    [
        ('vect', CountVectorizer(analyzer='word')),
        ('tfidf', TfidfTransformer()),    
    ]
)

In [17]:
df = preprocess_df(df)
add_sentiment_subj(df)
#df.describe()

In [18]:
X_train, X_test = train_test_split(df)


In [19]:
scored = X_train.loc[X_train['compound'] != 0]

In [20]:
X = pipeline.fit_transform(scored["text"])

## Thread Reconstruction

In [ ]:
mport anytree
from anytree import *
from anytree.search import find
from anytree import Node, RenderTree, AsciiStyle
import json
from anytree import AnyNode
from anytree.exporter import JsonExporter
from anytree.exporter import DotExporter
from anytree.exporter import DictExporter

def checkpar(x, lst):
    if (x.parent is not None):
        return x.parent.name in lst
    else:
        None

nodes = []
for i, row in df.iterrows():    
    clear_output(wait=True)
    print(i)
    foundNode = None
    thisNode = None
    myid = str(int(row['id']))
    searcharr = [myid]
    parent = None
    appended = False
    if not math.isnan(row["parent"]):
        parent = str(int(row["parent"]))
    if len(nodes) > 0:        
        for oldnode in nodes:
            #our guy
            foundNodes = findall(oldnode, filter_=lambda nd: checkpar(nd, searcharr))
            if len(foundNodes) > 0:
                if appended == False:
                    if parent is not None:
                        thisNode = Node(myid,parent=Node(parent))
                    else:
                        thisNode = Node(myid)
                    nodes.append(thisNode)
                    appended = True
                for node in foundNodes:
                        node.parent = thisNode                                    
    if appended == False:
        if parent is not None:
            thisNode = Node(myid,parent=Node(parent))
        else:
            thisNode = Node(myid)
        nodes.append(thisNode)



exporter = JsonExporter(indent=2)
for nd in nodes:
    if nd.height > 0:
        js  = exporter.export(nd)
        with open(f"./jsons/{root.name}.json", 'w') as outfile:
            json.dump(js, outfile)
        DotExporter(nd).to_picture(f"./jsons/pngs/{nd.name}.png")

16
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** found node for nan
** foun

KeyboardInterrupt: 

In [ ]:
from collections import defaultdict

ROOT = -1

def create_tree(pairs):
    """ Given an iterable of (parent, child) identifiers, build a tree
        where each node is a dict whose key is its identifier, and whose
        value is a list of children (which may be empty for leaf nodes).
        Multiple root nodes are supported; all roots are placed under a
        synthetic ROOT node.
    """
    def node(): return [ROOT,[]]
    table = defaultdict(node)
    # Build 2-way mapping between nodes
    idx = 0
    for parent, child in pairs:
        idx = idx+1
        clear_output(wait=True)
        print(idx)
        table[parent][1].append(child)  # parent - > children
        table[child][0] = parent  # child -> parent

    def follow(parent, childids):
        for c in childids:
            empty = []
            child = {c: empty}
            parent.append(child)
            if c in table:
                follow(empty, table[c][1])
                
    # Recursively fill in the tree
    tree = {ROOT:[]}
    roots = [k for k,v in table.items() if v[0] == ROOT]
    follow(tree[ROOT], roots)   
    return tree

#pairs = [(1,2),(2,3),(2,4),(1,5),(6,7),(6,8),(8,9)]


In [ ]:
import math
pairs = []
for i, row in df.iterrows():  
    myid = str(int(row['id']))
    parent = None
    if not math.isnan(row["parent"]):
        parent = str(int(row["parent"]))
    pair = (myid, parent)
    pairs.append(pair)

In [ ]:
create_tree(pairs)

NameError: name 'clear_output' is not defined

## Counting from DF / users

In [82]:
df2 = df.groupby('by')['id'].apply(list)


In [86]:
user = pd.DataFrame()
user["by"] = df['by']


In [88]:
user["comment_ids"], user["num_comments"] = [[[]] * user.shape[0], [[]] * user.shape[0]]
for i, row in user.iterrows():
  lst = list(df2[row["by"]])
  user.at[i,"comment_ids"] = lst
  user.at[i, "num_comments"] = len(df2[row["by"]])


In [91]:
user.sort_values(by=['num_comments'], ascending=False)
users = user.drop_duplicates()

TypeError: ignored

In [92]:
user2 = user.loc[user.astype(str).drop_duplicates().index]

In [95]:
user2 = user2.sort_values(by=['num_comments'], ascending=False)
user2.head()


,by,comment_ids,num_comments
614,tptacek,"[5294759, 498424, 5473500, 7615911, 1089926, 1402981, 1584288, 1345159, 2538143, 4720061, 1527080, 8041917, 7967104, 1768803, 3530003, 1808129, 13395642, 2855761, 8591353, 1739306, 4231075, 14390029, 11715716, 5862478, 10449240, 422388, 6471821, 3405992, 10657489, 3506564, 1123263, 6020093, 6630646, 6424527, 4678531, 483057, 363501, 13237370, 5683896, 7598759, 1372514, 8285584, 8062304, 9627094, 10422277, 1660903, 1623530, 7560676, 4342892, 1149002, 5039862, 9376263, 7959239, 10571552, 4071235, 1447854, 6044659, 397620, 2910179, 589948, 9505698, 5194771, 5340672, 10487811, 13208799, 10352115, 488337, 890600, 9951580, 7252610, 569598, 2357986, 5986572, 4091791, 6065143, 4685201, 1208402, 4415979, 6296698, 14390558, 7139511, 8489712, 9482047, 8964615, 2273789, 4138549, 532559, 8857098, 3853425, 990179, 13084827, 3359494, 5996025, 13635681, 4464049, 1578884, 997873, 1063508, 5616627, 6017491, ...]",279
33,jacquesm,"[10687435, 737348, 1289561, 871846, 8879714, 829858, 798440, 1973835, 1505851, 9838654, 11974192, 1555800, 14087505, 10563694, 14214119, 4854809, 12093396, 10694847, 1376162, 1341727, 4828349, 12153314, 2152809, 1800724, 1652935, 12451616, 10686220, 1780106, 14593733, 5178702, 1961727, 10661374, 13548703, 690215, 749393, 7864422, 1680210, 11305442, 871091, 719213, 871367, 1330162, 1102962, 715172, 5041879, 8870961, 13940637, 6452381, 14449302, 878055, 9582199, 10780156, 9743413, 9663891, 6287371, 14346663, 777959, 1341491, 857148, 10767793, 886432, 5852932, 4976150, 11880950, 760270, 9063681, 14101410, 1716953, 897994, 8201879, 6458235, 10262047, 1209904, 719190, 5012257, 10788682, 14077554, 10915222, 7889171, 12412050, 465868, 8256335, 9599321, 1790618, 5257184, 8344057, 1382786, 942130, 11215860, 1383733, 829207, 1360473, 7921250, 9287109, 10932876, 10635689, 14481004, 818465, 12150579, 1372955, ...]",184
153,DanBC,"[14576226, 9973871, 10502216, 12127927, 9144912, 7159701, 10065414, 11268265, 7372096, 5190692, 3889792, 7456101, 5553827, 8542766, 6839354, 7501144, 8089121, 4992539, 5121132, 10683724, 11682574, 13667355, 10203221, 3964160, 5457421, 3146919, 5368695, 4962571, 10919594, 3173897, 10703033, 10439721, 5088989, 9106332, 9898562, 7270965, 12082382, 3991349, 14458764, 7264221, 5442721, 8577521, 10855682, 3801860, 6815292, 3594563, 6495436, 6497190, 5531549, 10809876, 9288566, 3812573, 4010116, 3831985, 4232315, 11094190, 9010737, 6965227, 8147076, 10328712, 6464754, 3893072, 9968373, 13052514, 13744344, 9603723, 12065303, 3433498, 8851874, 10426174, 13293933, 4740679, 13397572, 4754191, 9281701, 9567026, 3595154, 12782990, 7990283, 5472474, 8468128, 9337598, 10567297, 6193357, 9550668, 8346700, 8339703, 5115779, 5514038, 7146321, 7946361, 8868027, 5996916, 12427213, 5231813, 10151607, 5778226, 14488963, 6327328, 6612433, ...]",141
786,mikeash,"[6233212, 10221042, 8490940, 3166549, 8957182, 11381227, 6548790, 13820769, 10582020, 12124917, 8832134, 8393780, 12922863, 9078640, 11389015, 8957383, 7883452, 7223050, 8092015, 10912077, 14220522, 11084948, 3082112, 8036467, 4058062, 7921763, 3613061, 13350377, 10143887, 12893126, 4191404, 10569876, 6015063, 10851070, 10509450, 10778591, 6687081, 13027313, 14033327, 10508895, 10131549, 6420077, 9949394, 9460179, 8729656, 4558988, 3881011, 3956851, 12015476, 7784976, 4350586, 7668433, 12952467, 6015019, 10062608, 5769590, 8170108, 6263960, 11340797, 13785198, 10832771, 7619947, 10941794, 9506547, 6826018, 4299633, 9377603, 10728876, 11125951, 10535271, 8410624, 7644223, 10304370, 11280466, 9787500, 7127285, 7861048, 14255643, 11224361, 8438411, 10572834, 7406621, 4695695, 5564691, 11001783, 7208912, 7916032, 9341432, 7419071, 6848353, 4345773, 4364579, 12139309, 5677299, 11901488, 10073884, 5322648, 4045656, 10496382, 8754773, ...]",115
213,icebraining,"[9711798, 12868517, 3657609, 4758963, 5194129, 6327635, 10951666, 8624810, 10618202, 6711653, 9684764, 9201022, 8393299, 7176093, 8625028, 9126647, 3646056, 10872211, 8247458, 32

In [111]:
user2["compound"] = np.nan
for i, row in user2.iterrows():  
    score  = df.loc[df['id'].isin(row["comment_ids"])]["compound"].mean()
    user2.at[i,"compound"] = score


Streaming output truncated to the last 5000 lines.
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

In [113]:
user2 = user2.sort_values(by=['compound'], ascending=True)
user2.head()




,by,comment_ids,num_comments,compound
26616,erklik,[12909755],1,-0.9995
30185,hopeless_case,[5414518],1,-0.9993
49716,schappim,[6995579],1,-0.9970
60832,ihsanshanti,[1059676],1,-0.9959
28799,JamisonM,"[5863401, 3332281, 4963102]",3,-0.9943


In [118]:
saltiest = user2.loc[(user2["num_comments"] > 10) & (user2["compound"] < -0.2)].sort_values(by=['compound'], ascending=True)

In [133]:
pd.set_option(
    'display.max_colwidth', 40
)

for i, row in saltiest.iterrows():  
    print(f"Showing {row['by']} with total score of {row['compound']}\n")    
    comments = df.loc[df['id'].isin(row["comment_ids"])]
    worst_comment = comments[comments["compound"] == comments.compound.min()]
    for j, row2 in worst_comment.iterrows():
      print(f"{row2['text']}\n")
    print("***\n")
    

Showing electromagnetic with total score of -0.3951

> What I was referring to was the Israeli Law of Return that gives citizenship to any Jew that applies.So the country established by all the western powers as a home for Judaism shouldn't permit Jews citizenship based on religion? You're an idiot and you can't comprehend it and are publicly flaunting it.Seriously read a book with a viewpoint not espoused by a neo-nazi group and you might actually learn something about the Israel-Palestine issue.Israel isn't the only country that's actively discriminating against Arabs they're just the easy one to point to. Let's forget that the Caucasus are in the middle of a religious war led by Arab religious sects attempting a religious coup in many parts of Russia. Let's forget that the Jews have been persecuted for thousands of years and Israel is surrounded by people trying to kill them simply because their Jews. Hey no shit those ancestors of the jews who survived all their families being kill

## Decision Tree

In [ ]:
# pipeline = Pipeline(
#     [
#         ('vect', CountVectorizer(analyzer='word')),
#         ('tfidf', TfidfTransformer()),    
#     ]
# )

In [ ]:
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X, scored["compound"])

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [ ]:
scored_test = X_test.loc[X_test['compound'] != 0]
X2 = pipeline.transform(scored_test["text"])
print(scored_test["compound"].shape)
print(X2.shape)

In [ ]:
from sklearn.metrics import mean_squared_error
predictions = tree_reg.predict(X2)

tree_mse = mean_squared_error(scored_test["compound"], predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.6365505601380854

## Keras 

In [21]:

EPOCHS = 10

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[X.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [22]:
model = build_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                4887488   
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 4,891,713
Trainable params: 4,891,713
Non-trainable params: 0
_________________________________________________________________


In [23]:
checkpoint_path = "./keras/"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
history = model.fit(
  X, scored["compound"],
  epochs=EPOCHS, validation_split = 0.2, verbose=1, callbacks=[cp_callback],)

Train on 41558 samples, validate on 10390 samples
Epoch 1/10
41536/41558 [============================>.] - ETA: 0s - loss: 0.1815 - mae: 0.3302 - mse: 0.1815
Epoch 00001: saving model to ./keras/
41558/41558 [==============================] - 125s 3ms/sample - loss: 0.1816 - mae: 0.3302 - mse: 0.1816 - val_loss: 0.1444 - val_mae: 0.2914 - val_mse: 0.1444
Epoch 2/10
41536/41558 [============================>.] - ETA: 0s - loss: 0.1077 - mae: 0.2361 - mse: 0.1077
Epoch 00002: saving model to ./keras/
41558/41558 [==============================] - 98s 2ms/sample - loss: 0.1078 - mae: 0.2362 - mse: 0.1078 - val_loss: 0.1421 - val_mae: 0.2694 - val_mse: 0.1421
Epoch 3/10
41536/41558 [============================>.] - ETA: 0s - loss: 0.0819 - mae: 0.1983 - mse: 0.0819
Epoch 00003: saving model to ./keras/
41558/41558 [==============================] - 112s 3ms/sample - loss: 0.0819 - mae: 0.1983 - mse: 0.0819 - val_loss: 0.1387 - val_mae: 0.2604 - val_mse: 0.1387
Epoch 4/10
41536/41558 [===

In [ ]:
def predict_text(text, model = model):
    xtemp = pipeline.transform([text])
    predictions = model.predict(xtemp)
    print(predictions)
    return predictions



txt = "Reality is that while China blocks Facebook, Google, etc and smartly props up their own clones, it’s “aghast” at the American protectionism and xenophobic behaviour. How dare the Americans block a Chinese app?! China is not a democracy. It’s not interested in fairness. China is playing the long game. Just like the wars of the past were fought with little toy armies of a few thousand knights and noblemen marching into each other’s countries until someone decided to conscript their whole nation into battle, the West is fighting allowing China to pilfer its technology, wreak the environment, and compete with state backed organisations. Wanna compete with Huawei? Good luck sending in your company noblemen, China is sending their whole nation behind it."


preds = predict_text("Impressive! Personally, I am a few thousand places behind, but still in the top 0.2%. How? I asked and answered a few hundred questions early on, years ago, when SO was new and interesting... Now those answers are old and, like most answers on SO, out of date. Usually when I google something technical and get seemingly the exact right question asked on SO, the answers are no longer correct. Software versions change. And yet I still get a steady trickle of votes, forever increasing the gulf between outdated and no-longer-participating people like me and anyone starting on SO today.")


[[0.6620629]]


In [ ]:
predictions = model.predict(X2)


In [ ]:
tree_mse = mean_squared_error(scored_test["compound"], predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.39776707368686554

[nltk_data] Downloading package subjectivity to
[nltk_data]     /Users/stepanboltalin/nltk_data...
[nltk_data]   Unzipping corpora/subjectivity.zip.
Cannot find the sentiment analyzer you want to load.
Training a new one using NaiveBayesClassifier.
Training classifier
Most Informative Features
            contains(--) = True             subj : obj    =     61.0 : 1.0
        contains(film's) = True             subj : obj    =     33.7 : 1.0
       contains(decides) = True              obj : subj   =     28.3 : 1.0
      contains(discover) = True              obj : subj   =     26.3 : 1.0
    contains(girlfriend) = True              obj : subj   =     25.0 : 1.0
          contains(town) = True              obj : subj   =     23.4 : 1.0
  contains(entertaining) = True             subj : obj    =     22.2 : 1.0
     contains(detective) = True              obj : subj   =     21.0 : 1.0
        contains(mother) = True              obj : subj   =     20.1 : 1.0
         contains(finds) = Tru

subj
